In [8]:
using Random

In [12]:
function get_instancia(n)
    return rand(1:100, n)
end

function get_permutacoes(instancia)
    return [shuffle(instancia) for _=1:20]
end

get_permutacoes (generic function with 1 method)

In [16]:
function get_custo(instancia, tamanho)
    if tamanho == 1
        return instancia[1]
    else
        return instancia[tamanho] + get_custo(instancia[1:tamanho], tamanho-1)
    end
end

get_custo (generic function with 1 method)

In [13]:
instancia = get_instancia(10)

10-element Vector{Int64}:
  26
  93
 100
  41
  61
  83
  62
  11
  19
  59

In [14]:
permutacoes = get_permutacoes(instancia)

20-element Vector{Vector{Int64}}:
 [61, 19, 100, 93, 59, 11, 62, 41, 26, 83]
 [41, 19, 61, 93, 83, 11, 62, 59, 100, 26]
 [59, 11, 83, 26, 62, 61, 19, 41, 100, 93]
 [11, 19, 61, 41, 62, 93, 83, 100, 59, 26]
 [61, 59, 26, 93, 62, 11, 83, 41, 19, 100]
 [100, 83, 26, 41, 62, 11, 19, 59, 93, 61]
 [19, 93, 11, 41, 62, 59, 83, 26, 100, 61]
 [59, 62, 41, 83, 26, 93, 19, 11, 100, 61]
 [11, 83, 61, 26, 93, 19, 62, 100, 59, 41]
 [83, 100, 59, 19, 11, 62, 26, 61, 93, 41]
 [59, 41, 100, 93, 11, 26, 19, 61, 62, 83]
 [61, 83, 11, 62, 93, 59, 19, 41, 26, 100]
 [41, 61, 59, 19, 62, 26, 11, 83, 100, 93]
 [100, 26, 41, 11, 59, 83, 62, 93, 61, 19]
 [26, 100, 93, 19, 41, 62, 61, 11, 83, 59]
 [61, 26, 11, 41, 62, 93, 59, 19, 83, 100]
 [59, 62, 61, 19, 26, 11, 93, 100, 41, 83]
 [83, 62, 26, 59, 41, 19, 100, 11, 61, 93]
 [26, 100, 11, 19, 59, 83, 41, 61, 93, 62]
 [61, 59, 11, 83, 26, 41, 93, 100, 62, 19]

In [17]:
for permutacao in permutacoes
    println(get_custo(permutacao, 10))
end

555
555
555
555
555
555
555
555
555
555
555
555
555
555
555
555
555
555
555
555


In [60]:
permutacoes

10×20 Matrix{Int64}:
 32  65   1   1  75  75   1  31  30  …  30  31  32  31  75  58   1  31  57
 32  31  30  58  30  65  32  40  30     58  31  40  31  31  58  32  30  32
 57  31  31  17  65  58  57  75  32     57  58  75  65  57  65  17  58   1
 32  30  31  40  30  30  65  58  58      1  65  30  40  65   1  65  17  30
 32   1  58  40  32  57  31  65  30     32  58   1  17  32  30  32  65  31
 17  57   1  17  65  17  17  32  17  …  30  65  40  32  57  31  31  32  32
 75  30   1  65  31  30  32  30  31     40  65  30  32  17  65  32  30  57
 65  57  57  32  17  75   1  40  58     57   1  57  32  40  65  31  65  31
 17  31  75  30  57  17   1  31  58     30   1  65  75  57  58  30  58  58
 30  75  57  65  17  75  58  17   1     58  30  65   1  30  17  40  40  31